In [ ]:
# Import minimal requirements
import requests
import json
import re

In [ ]:
# Set the base URL for the ARAX reasoner and its endpoint
endpoint_url = 'https://arax.rtx.ai/api/rtx/v1/query'

# Create a dict of the request, specifying the list of DSL commands
query = {"previous_message_processing_plan": {"processing_actions": [
            "add_qnode(name=DOID:9406, key=n00)",
            "add_qnode(type=biolink:ChemicalSubstance, is_set=true, key=n01)",
            "add_qnode(type=biolink:Protein, key=n02)",
            "add_qedge(subject=n00, object=n01, key=e00)",  
            "add_qedge(subject=n01, object=n02, key=e01)",
            "expand(edge_key=[e00,e01])", 
            "overlay(action=overlay_clinical_info, observed_expected_ratio=true, virtual_relation_label=C1, subject_qnode_key=n00, object_qnode_key=n01)", 
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=observed_expected_ratio, direction=below, threshold=3, remove_connected_nodes=t, qnode_id=n01)",   
            "filter_kg(action=remove_orphaned_nodes, node_type=protein)",
            "overlay(action=compute_ngd, virtual_relation_label=N1, subject_qnode_key=n01, object_qnode_key=n02)", 
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=ngd, direction=above, threshold=0.85, remove_connected_nodes=t, qnode_id=n02)",
            "resultify(ignore_edge_direction=true)",
            "return(message=true, store=true)"
        ]}}

In [ ]:
# Send the request to RTX and check the status
print(f"Executing query at {endpoint_url}\nPlease wait...")
response_content = requests.post(endpoint_url, json=query, headers={'accept': 'application/json'})
status_code = response_content.status_code
if status_code != 200:
    print("ERROR returned with status "+str(status_code))
    print(response_content.json())
else:
    print(f"Response returned with status {status_code}")

In [ ]:
# Unpack respsonse from JSON and display the information log
response_dict = response_content.json()
for message in response_dict['log']:
    if message['level'] >= 20:
        print(message['prefix']+message['message'])

In [ ]:
# These URLs provide direct access to resulting data and GUI
if 'id' in response_dict and response_dict['id'] is not None:
    print(f"Data: {response_dict['id']}")
    match = re.search(r'(\d+)$', response_dict['id'])
    if match:
        print(f"GUI: https://arax.rtx.ai/?m={match.group(1)}")
else:
    print("No id was returned in response")

In [ ]:
# Or you can view the entire Translator API response Message
print(json.dumps(response_dict, indent=2, sort_keys=True))